In [1]:
import os
import open3d as o3d
import time
from pathlib import Path

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
#IMPORT MODULES
from context import geomapi 
from geomapi.nodes import *
import geomapi.utils as ut
from geomapi.utils import geometryutils as gmu
import geomapi.tools as tl
from geomapi.tools import validationtools as vt
from geomapi.tools import completiontools as ct

c:\Users\SamDeGeyter\anaconda3\envs\geomapi\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
path = r"C:\Data\Ramen\Ramen"
print(os.listdir(path))

['Detail', 'Sparse', 'Window 1.e57', 'Window 1.obj', 'Window 1.pcd', 'Window 10.e57', 'Window 10.obj', 'Window 10.pcd', 'Window 2.e57', 'Window 2.obj', 'Window 2.pcd', 'Window 3.e57', 'Window 3.obj', 'Window 3.pcd', 'Window 4.e57', 'Window 4.obj', 'Window 4.pcd', 'Window 5.e57', 'Window 5.obj', 'Window 5.pcd', 'Window 6.e57', 'Window 6.obj', 'Window 6.pcd', 'Window 7.e57', 'Window 7.obj', 'Window 7.pcd', 'Window 8.e57', 'Window 8.obj', 'Window 8.pcd', 'Window 9.e57', 'Window 9.obj', 'Window 9.pcd']


In [4]:
BIMNodes = []
PCDNodes = []

for window in os.listdir(path):
    if window.endswith("obj"):
        bimNode = BIMNode(path = os.path.join(path, window))
        bimNode.get_resource()
        BIMNodes.append(bimNode)
        pcdPath = os.path.join(path, window.split(".")[0] + ".pcd")
        pcdNode = PointCloudNode(path = pcdPath)
        # pcdNode = tl.e57path_to_nodes_mutiprocessing(pcdPath)[0]
        pcdNode.get_resource()
        PCDNodes.append(pcdNode)
        bimNode.linkedNodes = [pcdNode]

In [5]:
# geometries = [PCDNodes[1].resource, BIMNodes[1].resource]
# o3d.visualization.draw_geometries(geometries, mesh_show_wireframe=True)

In [6]:
#Create pcds from the meshes
BIMPCDNodes = []
resolution = 0.005
for bimNode in BIMNodes:
    density=int(bimNode.resource.get_surface_area()/(resolution*resolution))
    bimpcd=bimNode.resource.sample_points_uniformly(number_of_points=density)

    bimpcdNode = PointCloudNode(resource = bimpcd)
    bimpcdNode.linkedNodes = bimNode.linkedNodes
    BIMPCDNodes.append(bimpcdNode)
    

In [7]:
print("BIMNodes: %s" %len(BIMNodes))
print("PCDNodes: %s" %len(PCDNodes))
print("BIMPCDNodes: %s" %len(BIMPCDNodes))

BIMNodes: 10
PCDNodes: 10
BIMPCDNodes: 10


In [8]:
for pointcloudNode in PCDNodes:
    target = pointcloudNode.resource
    T = target.get_center() * -1
    pointcloudNode.resource = target.translate(T)

for bimpcdNode in BIMPCDNodes:
    #compare all pointclouds with the first BIMNode
    source = bimpcdNode.resource
    #Move the BIMNode to the origin
    T = source.get_center() * -1
    bimpcdNode.resource = source.translate(T)

In [9]:
# #Reduce pointclouds to a distance of 0.5m of the mesh object
# import numpy as np
# td = 0.5
# count = 1
# for bimpcdNode in BIMPCDNodes:
#     # print(bimpcdNode.linkedNodes[0].resource)
#     source = bimpcdNode.linkedNodes[0].resource
#     c2cdistances = source.compute_point_cloud_distance(bimpcdNode.resource)
#     ids = np.where(np.asarray(c2cdistances) <= td)[0]
#     bimpcdNode.linkedNodes[0].resource = source.select_by_index(ids)
#     # print(bimpcdNode.linkedNodes[0].resource)
#     bimpcdNode.linkedNodes[0].name = "Window " + str(count)
#     bimpcdNode.linkedNodes[0].save_resource(path)
#     count += 1


In [10]:
# # Search distance for Nearest Neighbour Search [Hybrid-Search is used].
# max_correspondence_distance = 0.07

# # Select the `Estimation Method`, and `Robust Kernel` (for outlier-rejection).
# estimation = o3d.t.pipelines.registration.TransformationEstimationPointToPlane()

# # Convergence-Criteria for Vanilla ICP:

# criteria = o3d.t.pipelines.registration.ICPConvergenceCriteria(relative_fitness=0.000001,
#                                        relative_rmse=0.000001,
#                                        max_iteration=50)

# # Vanilla ICP
# voxel_size = 0.01

# # Save iteration wise `fitness`, `inlier_rmse`, etc. to analyse and tune result.
# save_loss_log = True

In [11]:

# for bimpcdNode in BIMPCDNodes:
#     #compare all pointclouds with the first BIMNode
#     bimpcd = bimpcdNode.resource
#     #Move the BIMNode to the origin
#     T = bimpcd.get_center() * -1
#     bimpcd.translate(T)

#     bimpcd.paint_uniform_color([0,0,1])
#     source = o3d.t.geometry.PointCloud.from_legacy(bimpcd)
#     source.estimate_normals()
#     source.point["colors"] = source.point["colors"].to(
#         o3d.core.Dtype.Float32) / 255.0

#     ptime = []
#     fitness = []
#     rmse = []

#     for pointcloudNode in PCDNodes:
#         pointcloud = pointcloudNode.resource
#         T = pointcloud.get_center() * -1
#         pointcloud.translate(T)
#         pointcloud.paint_uniform_color([0,0,1])
#         target = o3d.t.geometry.PointCloud.from_legacy(pointcloud)
#         # Initial guess transform between the two point-cloud.
#         # ICP algortihm requires a good initial allignment to converge efficiently.
#         trans_init = np.asarray([[0.862, 0.011, -0.507, 0.5],
#                                 [-0.139, 0.967, -0.215, 0.7],
#                                 [0.487, 0.255, 0.835, -1.4], [0.0, 0.0, 0.0, 1.0]])
        
#         target.transform(o3d.core.Tensor(trans_init))
#         target.point["colors"] = target.point["colors"].to(
#         o3d.core.Dtype.Float32) / 255.0
#         s = time.time()

#         registration_icp = o3d.t.pipelines.registration.icp(source, target, max_correspondence_distance,
#                                     trans_init, estimation, criteria,
#                                     voxel_size)

#         icp_time = time.time() - s
#         ptime.append(icp_time)
#         # print("Time taken by ICP: ", icp_time)
#         fitness.append(registration_icp.fitness)
#         # print("Inlier Fitness: ", registration_icp.fitness)
#         rmse.append(registration_icp.inlier_rmse)
#         # print("Inlier RMSE: ", registration_icp.inlier_rmse)
#     # print(ptime)
#     print(fitness)
#     # print(rmse)


[0.9841238111270095, 0.5435358976517128, 0.7724097298207222, 0.9415768874673849, 0.9518874673849003, 0.7065167073478663, 0.6275460819796314, 0.3595656931234745, 0.664516875683865, 0.7967658446258733]
[1.0, 0.9883616255144033, 0.9533822016460906, 0.9844393004115226, 0.9985210905349794, 0.8826517489711934, 0.9969778806584362, 0.7231224279835391, 1.0, 0.9870756172839507]
[1.0, 0.8082275552029617, 0.8857926748644718, 0.9656386354621183, 0.9718200449557054, 0.8035832341663361, 0.778758429194764, 0.5046608488694962, 0.9460696813433822, 0.7846092820309402]
[1.0, 0.8647498443014324, 0.8401494706248702, 0.987051069130164, 0.9991436578783475, 0.9161044218393191, 0.978202200539755, 0.6188239568195972, 0.9548474154037783, 0.8336360805480589]
[1.0, 0.8743039752094127, 0.817145208928485, 0.984360625574977, 0.9992010845881955, 0.9174211978889265, 0.9635161961942575, 0.6228392969544376, 0.9643151116060621, 0.8252796203941316]
[1.0, 0.9293430888296046, 0.8957183232520669, 0.9833857207004939, 0.9971208959560475, 0.9180908106394781, 0.9215246044533665, 0.6555376528698592, 0.9750653741514568, 0.9017142555270874]
[1.0, 0.9967022010890406, 1.0, 0.9744612316895468, 0.9999233070020707, 0.942940409540609, 1.0, 0.9410230845923767, 1.0, 0.9934044021780811]
[1.0, 0.8446231429739676, 0.9918222706828403, 0.9880741447458089, 0.9987051928581164, 0.7582799509336241, 1.0, 0.7950115851165327, 0.9670846394984326, 0.9905274635409568]
[1.0, 0.9968744573710714, 0.969091856225039, 0.9816808473693349, 0.9999131793714187, 0.9546796318805348, 1.0, 0.7990102448341726, 1.0, 0.9842854662267755]
[0.9712598217581606, 0.5534672783842219, 0.8057585825027685, 0.9672731107947055, 0.9186415651531931, 0.6945420028476507, 0.48527131782945737, 0.3014712861888942, 0.6327268892052945, 0.7956757896957233]

Global Registration

In [12]:
# import copy 
# trans_init = np.asarray([[0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0],
#                              [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0]])

# voxel_size = 0.01
# distance_threshold = voxel_size * 1.5

# for bimpcdNode in BIMPCDNodes:
#     #compare all pointclouds with the first BIMNode
#     source = bimpcdNode.resource
#     #Move the BIMNode to the origin
#     T = source.get_center() * -1
#     source.translate(T)
#     source.paint_uniform_color([0,0,1])

#     source.transform(trans_init)

#     source_down = source.voxel_down_sample(voxel_size)
#     radius_normal = voxel_size * 2
#     source_down.estimate_normals(
#         o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))
    
#     radius_feature = voxel_size * 5
#     source_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
#         source_down,
#         o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    
#     ptime = []
#     fitness = []
#     rmse = []


#     for pointcloudNode in PCDNodes:
#         target = pointcloudNode.resource
#         T = target.get_center() * -1
#         target.translate(T)
#         target.paint_uniform_color([0,0,1])

#         target_down = target.voxel_down_sample(voxel_size)
#         radius_normal = voxel_size * 2
#         target_down.estimate_normals(
#             o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))
        
#         radius_feature = voxel_size * 5
#         target_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
#             target_down,
#             o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
#         s = time.time()
#         result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
#             source_down, target_down, source_fpfh, target_fpfh, True,
#             distance_threshold,
#             o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
#             3, [
#                 o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(
#                     0.9),
#                 o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(
#                     distance_threshold)
#             ], o3d.pipelines.registration.RANSACConvergenceCriteria(100000, 0.999))
#         # print(result)
#         icp_time = time.time() - s
#         ptime.append(icp_time)
#         # print("Time taken by ICP: ", icp_time)
#         fitness.append(result.fitness)
#         # print("Inlier Fitness: ", registration_icp.fitness)
#         rmse.append(result.inlier_rmse)
#         # print("Inlier RMSE: ", registration_icp.inlier_rmse)
#     # print(ptime)
#     print(fitness)
#     # print(rmse)

Histograms

In [13]:
# import matplotlib.pyplot as plt
# import numpy as np

# for i, bimpcdNode in enumerate(BIMPCDNodes):
#     color = 'blue'
#     #compare all pointclouds with the first BIMNode
#     source = bimpcdNode.resource
#     #Move the BIMNode to the origin
#     Xbimpcd = np.asarray(source.points)[:,0]

#     n, bins, patches = plt.hist(x=Xbimpcd, bins='auto', color=color,
#                             alpha=0.7, rwidth=0.90, density=True)
#     plt.show()

#     for j, pointcloudNode in enumerate(PCDNodes):
#         if j == i:
#             color = ('green')
#             target = pointcloudNode.resource
#             Xtarget = np.asarray(target.points)[:,0]
#             n, bins2, patches = plt.hist(x=Xtarget, bins=bins, color=color,
#                                     alpha=0.7, rwidth=0.90, density=True)
            
#             plt.show()
#         else:
#             color = 'red'
#         # target = pointcloudNode.resource
#         # Xtarget = np.asarray(target.points)[:,0]
#         # n, bins2, patches = plt.hist(x=Xtarget, bins=bins, color=color,
#         #                         alpha=0.7, rwidth=0.90, density=True)
        
#         # plt.show()
#             # plt.grid(axis='y', alpha=0.75)
#             # plt.xlabel('Value')
#             # plt.ylabel('Frequency')
#             # plt.title('My Very Own Histogram')
#             # plt.text(23, 45, r'$\mu=15, b=3$')
#             # maxfreq = n.max()
#             # Set a clean upper y-axis limit.
#             # plt.ylim(ymax=np.ceil(maxfreq / 10) * 10 if maxfreq % 10 else maxfreq + 10)
    
      

In [14]:
# import matplotlib.pyplot as plt
# import numpy as np

# for i, bimpcdNode in enumerate(BIMPCDNodes):
#     color = 'blue'
#     #compare all pointclouds with the first BIMNode
#     source = bimpcdNode.resource
#     #Move the BIMNode to the origin
#     Ybimpcd = np.asarray(source.points)[:,1]

#     n, bins, patches = plt.hist(x=Ybimpcd, bins='auto', color=color,
#                             alpha=0.7, rwidth=0.90, density=True)
#     plt.show()

#     for j, pointcloudNode in enumerate(PCDNodes):
#         if j == i:
#             color = ('green')
#             target = pointcloudNode.resource
#             Ytarget = np.asarray(target.points)[:,1]
#             n, bins2, patches = plt.hist(x=Ytarget, bins=bins, color=color,
#                                     alpha=0.7, rwidth=0.90, density=True)
            
#             plt.show()
#         else:
#             color = 'red'
#         # target = pointcloudNode.resource
#         # Ytarget = np.asarray(target.points)[:,1]
#         # n, bins2, patches = plt.hist(x=Ytarget, bins=bins, color=color,
#         #                         alpha=0.7, rwidth=0.90, density=True)
        
#         # plt.show()
#             # plt.grid(axis='y', alpha=0.75)
#             # plt.xlabel('Value')
#             # plt.ylabel('Frequency')
#             # plt.title('My Very Own Histogram')
#             # plt.text(23, 45, r'$\mu=15, b=3$')
#             # maxfreq = n.max()
#             # Set a clean upper y-axis limit.
#             # plt.ylim(ymax=np.ceil(maxfreq / 10) * 10 if maxfreq % 10 else maxfreq + 10)
    
      

In [15]:
# import matplotlib.pyplot as plt
# import numpy as np

# for i, bimpcdNode in enumerate(BIMPCDNodes):
#     color = 'blue'
#     #compare all pointclouds with the first BIMNode
#     source = bimpcdNode.resource
#     #Move the BIMNode to the origin
#     Zbimpcd = np.asarray(source.points)[:,2]

#     n, bins, patches = plt.hist(x=Zbimpcd, bins='auto', color=color,
#                             alpha=0.7, rwidth=0.90, density=True)
#     plt.show()

#     for j, pointcloudNode in enumerate(PCDNodes):
#         if j == i:
#             color = ('green')
#             target = pointcloudNode.resource
#             Ztarget = np.asarray(target.points)[:,2]
#             n, bins2, patches = plt.hist(x=Ztarget, bins=bins, color=color,
#                                     alpha=0.7, rwidth=0.90, density=True)
            
#             plt.show()
#         else:
#             color = 'red'
#         # target = pointcloudNode.resource
#         # Ztarget = np.asarray(target.points)[:,2]
#         # n, bins2, patches = plt.hist(x=Ztarget, bins=bins, color=color,
#         #                         alpha=0.7, rwidth=0.90, density=True)
        
#         # plt.show()
#             # plt.grid(axis='y', alpha=0.75)
#             # plt.xlabel('Value')
#             # plt.ylabel('Frequency')
#             # plt.title('My Very Own Histogram')
#             # plt.text(23, 45, r'$\mu=15, b=3$')
#             # maxfreq = n.max()
#             # Set a clean upper y-axis limit.
#             # plt.ylim(ymax=np.ceil(maxfreq / 10) * 10 if maxfreq % 10 else maxfreq + 10)
    
      

In [16]:
from jakteristics import compute_features
import numpy as np

for bimpcd in BIMPCDNodes:
    features = compute_features(np.asarray(bimpcd.resource.points), search_radius=0.15)


In [19]:
print(len(features))


420992
